In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0)  # input=28 Output=26 
        self.conv2 = nn.Conv2d(16, 32, 3, padding=0) # input=26 Output=24 
        self.conv3 = nn.Conv2d(32, 32, 3, padding=0) # input=24 Output=22 
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(32, 32, 1, padding=0) # input=22 Output=11 
        self.conv5 = nn.Conv2d(32, 32, 3, padding=0) # input=11 Output=09 
        self.conv5 = nn.Conv2d(32, 32, 3, padding=0) # input=09 Output=07 
        self.conv6 = nn.Conv2d(32, 64, 3, padding=0) # input=07 Output=07 
        self.conv7 = nn.Conv2d(64, 10, 1, padding=0) # input=07 Output=07 
        self.conv8 = nn.Conv2d(10, 10, 7, padding=0) # input=07 Output=01 

    def forward(self, x):
        x = self.pool1(F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x)))))))
        x = F.relu(self.conv5(F.relu(self.conv4(x))))
        x = F.relu(self.conv7(F.relu(self.conv6(x))))
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 32, 24, 24]           4,640
            Conv2d-3           [-1, 32, 22, 22]           9,248
         MaxPool2d-4           [-1, 32, 11, 11]               0
            Conv2d-5           [-1, 32, 11, 11]           1,056
            Conv2d-6             [-1, 32, 9, 9]           9,248
            Conv2d-7             [-1, 64, 7, 7]          18,496
            Conv2d-8             [-1, 10, 7, 7]             650
            Conv2d-9             [-1, 10, 1, 1]           4,910
Total params: 48,408
Trainable params: 48,408
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.45
Params size (MB): 0.18
Estimated Total Size (MB): 0.64
---------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    print("Epochs", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

Epochs 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2532, Accuracy: 9254/10000 (93%)

Epochs 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0372, Accuracy: 9877/10000 (99%)

Epochs 14


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0433, Accuracy: 9863/10000 (99%)

Epochs 15


loss=0.004817510489374399 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.34it/s]



Test set: Average loss: 0.0385, Accuracy: 9880/10000 (99%)

